In [4]:
import yfinance as yf 
from yfinance import EquityQuery
import numpy as np
import pandas as pd
import time

size = 250
rsi_window = 14
roc_window = 5
volatility_window = 30

In [5]:
from yfinance import EquityQuery, screen
import pandas as pd

# Define your screen query (as you already have)
screen_query = EquityQuery("and", [
    EquityQuery("is-in", ["region", "us"]),
    EquityQuery("is-in", ["sector",
        "Technology",
        "Financial Services",
        "Consumer Cyclical",
        "Communication Services",
        "Basic Materials",
        "Industrials"
    ]),
    EquityQuery("GTE", ["eodprice", 0.50]),
    EquityQuery("GTE", ["avgdailyvol3m", 100000]),
    EquityQuery("LTE", ["short_percentage_of_float.value", 100])
])

batch_size = 250
offset = 0
all_stocks = []

while True:
    screener = screen(
        screen_query,
        sortField="avgdailyvol3m",
        sortAsc=False,
        size=batch_size,
        offset=offset
    )
    stocks = screener.get('quotes', [])
    if not stocks:
        break
    all_stocks.extend(stocks)
    offset += batch_size
    print(f"Loaded {len(all_stocks)} stocks")

# Convert to DataFrame
stocks = pd.DataFrame(all_stocks)
print(f"Total stocks loaded: {len(stocks)}")



Loaded 250 stocks
Loaded 500 stocks
Loaded 750 stocks
Loaded 1000 stocks
Loaded 1250 stocks
Loaded 1500 stocks
Loaded 1750 stocks
Loaded 2000 stocks
Loaded 2050 stocks
Total stocks loaded: 2050


In [6]:
stocks = pd.DataFrame(stocks)
stocks = stocks.drop(columns=[
    'esgPopulated',
    'cryptoTradeable',
    'hasPrePostMarketData',
    'firstTradeDateMilliseconds',
    'priceHint',
    'regularMarketDayHigh',
    'regularMarketDayRange',
    'regularMarketDayLow',
    'regularMarketPreviousClose',
    'messageBoardId',
    'fullExchangeName',
    'financialCurrency',
    'regularMarketOpen',
    'averageDailyVolume3Month',
    'averageDailyVolume10Day',
    'corporateActions',
    'fiftyTwoWeekLowChange',
    'fiftyTwoWeekLowChangePercent',
    'fiftyTwoWeekRange',
    'fiftyTwoWeekHighChange',
    'fiftyTwoWeekHighChangePercent',
    'fiftyTwoWeekChangePercent',
    'earningsTimestamp',
    'earningsTimestampStart',
    'earningsTimestampEnd',
    'earningsCallTimestampStart',
    'earningsCallTimestampEnd',
    'dividendRate',
    'trailingAnnualDividendYield',
    'marketState',
    'epsCurrentYear',
    'fiftyDayAverage',
    'fiftyDayAverageChange',
    'fiftyDayAverageChangePercent',
    'twoHundredDayAverage',
    'twoHundredDayAverageChange',
    'twoHundredDayAverageChangePercent','dividendYield','gmtOffSetMilliseconds','bid','ask','currency', 'quoteType','language','typeDisp','quoteSourceName','triggerable','customPriceAlertConfidence','sourceInterval','exchangeDataDelayedBy','exchangeTimezoneName','exchangeTimezoneShortName',
 'bidSize','askSize','market','isEarningsDateEstimate','trailingAnnualDividendRate','regularMarketTime','priceEpsCurrentYear','epsForward','forwardPE','tradeable','displayName','longName'
]
)
df = stocks

In [7]:
order = [
    # Identification
    'symbol',
    'shortName',
    'exchange',
    'region',
    
    # Market Price & Change
    'regularMarketPrice',
    'regularMarketChange',
    'regularMarketChangePercent',
    'regularMarketVolume',
    
    # Valuation Metrics
    'marketCap',
    'sharesOutstanding',
    'bookValue',
    'priceToBook',
    'trailingPE',
    'epsTrailingTwelveMonths',
    'averageAnalystRating',
    
    # Price Range
    'fiftyTwoWeekHigh',
    'fiftyTwoWeekLow',
    
    # Dividend Info
    'dividendDate'
]
df = df[order]
df


,symbol,shortName,exchange,region,regularMarketPrice,regularMarketChange,regularMarketChangePercent,regularMarketVolume,marketCap,sharesOutstanding,bookValue,priceToBook,trailingPE,epsTrailingTwelveMonths,averageAnalystRating,fiftyTwoWeekHigh,fiftyTwoWeekLow,dividendDate
0,NVDA,NVIDIA Corporation,NMS,US,135.13,-4.060000,-2.916870,323313507,3394506063872,24387600384,3.438,39.304830,44.893690,3.01,1.4 - Strong Buy,153.130,86.620,1.743552e+09
1,F,Ford Motor Company,NYQ,US,10.38,0.160000,1.565560,132007518,40640294912,3905700096,11.225,0.924722,8.173228,1.27,3.0 - Hold,14.850,8.440,1.748822e+09
2,TSLA,"Tesla, Inc.",NMS,US,346.46,-11.970000,-3.339560,122775188,1154487812096,3220960000,23.184,14.943927,203.799990,1.70,2.6 - Hold,488.540,167.410,NaN
3,LCID,"Lucid Group, Inc.",NMS,US,2.23,-0.180000,-7.468880,254714279,7351128064,3050259968,1.044,2.136015,NaN,-1.10,3.1 - Hold,4.430,1.930,NaN
4,PLTR,Palantir Technologies Inc.,NMS,US,131.78,9.460000,7.733810,182153344,298206265344,2262909952,2.299,57.320570,599.000000,0.22,3.1 - Hold,133.490,21.005,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,AMSF,"AMERISAFE, Inc.",NMS,US,47.47,0.310001,0.657340,83315,904317760,19050300,13.699,3.465216,19.141129,2.48,2.7 - Hold,60.240,41.970,1.750378e+09
2046,BFST,"Business First Bancshares, Inc.",NMS,US,23.79,-0.339998,-1.409030,86345,706444096,29695000,25.510,0.932575,9.790124,2.43,NaN,30.298,18.970,1.748650e+09
2047,AVNW,"Aviat Networks, Inc.",NMS,US,21.45,0.300001,1.418450,105919,272252000,12692400,20.179,1.062986,NaN,-0.19,1.4 - Strong Buy,31.740,12.960,NaN
2048,BGH,Barings Global Short Duration H,NYQ,US,14.80,0.020001,0.135321,65374,296951648,20064300,NaN,NaN,7.047619,2.10,NaN,16.340,12.520,1.519862e+09


In [ ]:
# Get the price history for the stocks, to do RSI, or MACD, options volatility, and kalman filter acceleration(or see tview workaround)
stocklist = df.symbol.tolist()
stocklist

# Get the price history for the stocks
price_data = yf.download(stocklist, period='14d', interval='1d')['Close']


# Calculate the daily returns
change_data = price_data[stocklist]/price_data[stocklist].shift(1)-1

# RSI
change_data
def rsi(price_data, period=rsi_window):
    delta = change_data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))
rsi_data = rsi(price_data)
rsi_data

rsi = round(rsi_data.iloc[-1],1)
df['RSI'] = rsi.values


## Rate of change 
window=5 
recent_avg = change_data.rolling(window=roc_window).mean()
earlier_avg = change_data.shift(window).rolling(window=roc_window).mean()

# Acceleration: difference between recent and earlier averages
accel = (recent_avg - earlier_avg) * 100
accel = round(accel.iloc[-1], 2)
df['ROC'] = accel.values


# Apply filters________________________________________________________________________________________________________________________________________________________

filterdf = df[(df['RSI'] < 60) ] # Filter for RSI < 60

filterdf



YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  2050 of 2050 completed
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_46825/3717644340.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RSI'] = rsi.values
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_46825/3717644340.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ROC'] = accel.values


,symbol,shortName,exchange,region,regularMarketPrice,regularMarketChange,regularMarketChangePercent,regularMarketVolume,marketCap,sharesOutstanding,bookValue,priceToBook,trailingPE,epsTrailingTwelveMonths,averageAnalystRating,fiftyTwoWeekHigh,fiftyTwoWeekLow,dividendDate,RSI,ROC
0,NVDA,NVIDIA Corporation,NMS,US,135.13,-4.060000,-2.916870,323313507,3394506063872,24387600384,3.438,39.304830,44.893690,3.01,1.4 - Strong Buy,153.130,86.620,1.743552e+09,36.6,0.67
1,F,Ford Motor Company,NYQ,US,10.38,0.160000,1.565560,132007518,40640294912,3905700096,11.225,0.924722,8.173228,1.27,3.0 - Hold,14.850,8.440,1.748822e+09,55.3,0.47
2,TSLA,"Tesla, Inc.",NMS,US,346.46,-11.970000,-3.339560,122775188,1154487812096,3220960000,23.184,14.943927,203.799990,1.70,2.6 - Hold,488.540,167.410,NaN,41.6,0.46
3,LCID,"Lucid Group, Inc.",NMS,US,2.23,-0.180000,-7.468880,254714279,7351128064,3050259968,1.044,2.136015,NaN,-1.10,3.1 - Hold,4.430,1.930,NaN,44.9,-3.37
4,PLTR,Palantir Technologies Inc.,NMS,US,131.78,9.460000,7.733810,182153344,298206265344,2262909952,2.299,57.320570,527.120000,0.25,3.1 - Hold,133.490,21.005,NaN,49.5,2.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,AMSF,"AMERISAFE, Inc.",NMS,US,47.47,0.310001,0.657340,83315,904317760,19050300,13.699,3.465216,19.141129,2.48,2.7 - Hold,60.240,41.970,1.750378e+09,55.3,0.96
2046,BFST,"Business First Bancshares, Inc.",NMS,US,23.79,-0.339998,-1.409030,86345,706444096,29695000,25.510,0.932575,9.790124,2.43,NaN,30.298,18.970,1.748650e+09,45.4,0.88
2047,AVNW,"Aviat Networks, Inc.",NMS,US,21.45,0.300001,1.418450,105919,272252000,12692400,20.179,1.062986,NaN,-0.19,1.4 - Strong Buy,31.740,12.960,NaN,49.3,0.45
2048,BGH,Barings Global Short Duration H,NYQ,US,14.80,0.020001,0.135321,65374,296951648,20064300,NaN,NaN,7.047619,2.10,NaN,16.340,12.520,1.519862e+09,44.7,0.94


#"""Steps:
#1. get options volatility, prices, data 
#2. get RSI, kalman filter acceleration  (done)
#indicators in screener: filter by RSI, filter by relative options volatility (vs historical volatility, as yfinance does not have historical options volatility data)
#"""
# other:
#take steps to pull all yahoo screen companies 

In [ ]:
filterstocks = filterdf.symbol.tolist()
price_datayear = yf.download(filterstocks, period='3mo', interval='1d')["Close"]
#^^^ fix 
# Historical volatiltiy calculation
# Next add the vol smile relative to HV 
def hv(price_datayear, window=volatility_window):
    log_returns = np.log(price_datayear / price_datayear.shift(1))
    volatility = log_returns.rolling(window=window).std() * np.sqrt(252)  # Annualized volatility
    return volatility

hvol = hv(price_datayear)
hvol

[*********************100%***********************]  2013 of 2013 completed


Ticker,AA,AAL,AAOI,AAON,AAP,AAPL,AB,ABCB,ABG,ABM,...,ZETA,ZG,ZH,ZIM,ZION,ZIP,ZKIN,ZM,ZTO,ZUMZ
Date,,,,,,,,,,,,,,,,,,,,,
2025-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-23,0.506467,0.417077,1.067710,0.425714,1.393981,0.356372,0.288530,0.249864,0.296103,0.219366,...,0.497449,0.330025,0.396517,0.740198,0.310893,0.991285,3.131040,0.221906,0.371971,0.530042
2025-05-27,0.495151,0.427492,1.057939,0.410058,1.394010,0.344716,0.264193,0.254801,0.295926,0.211151,...,0.500752,0.328819,0.395954,0.710838,0.313303,0.991000,3.163914,0.214868,0.358426,0.576419
2025-05-28,0.497425,0.431043,1.074343,0.423303,1.400477,0.338702,0.264035,0.263992,0.293954,0.211718,...,0.499105,0.323025,0.418678,0.714557,0.314921,0.995153,3.163904,0.215056,0.368538,0.576643


In [ ]:
filterstocks = filterdf.symbol.tolist()
#  1. Download price data (make sure columns are tickers)
price_datayear = yf.download(filterstocks, period='3mo', interval='1d')['Close']

# 2. Calculate HV
hvol_latest = price_datayear.apply(hv).iloc[-1]
hvol_latest.name = 'hvol'

# 3. Prepare for merge
filterdf['symbol'] = filterdf['symbol'].astype(str).str.upper()
hvol_latest.index = hvol_latest.index.astype(str).str.upper()

# 4. Merge
filterdf = filterdf.merge(hvol_latest, left_on='symbol', right_index=True, how='left')


NameError: name 'filterdf' is not defined

In [ ]:
import yfinance as yf
import pandas as pd

def find_closest_value(data_list, target_value):
    """Find the value in a list or Series that is closest to a given target value."""
    if len(data_list) == 0:
        return None
    return min(data_list, key=lambda x: abs(x - target_value))

def filter_special_expiry(expiry_list):
    """
    Returns a list of expiries filtered to only the 2nd lowest, middle, 3rd to last, and last.
    """
    sorted_expiries = sorted(expiry_list)
    length = len(sorted_expiries)
    if length == 0:
        return []
    values_to_keep = set()
    if length > 1:
        values_to_keep.add(sorted_expiries[1])  # 2nd lowest
    values_to_keep.add(sorted_expiries[length // 2])  # middle
    if length >= 3:
        values_to_keep.add(sorted_expiries[-3])  # 3rd to last
    values_to_keep.add(sorted_expiries[-1])  # last
    return list(values_to_keep)

def get_options(symbol):
    try:
        stock = yf.Ticker(symbol)
        options_chain = stock.options
        if not options_chain:
            return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest'])
        # Filter expiries first
        filtered_expiries = filter_special_expiry(options_chain)
        options_data = []
        current_price = stock.history(period='1d')['Close'].iloc[-1]
        for expiry in filtered_expiries:
            chain = stock.option_chain(expiry)
            for opt_type, df in [('call', chain.calls), ('put', chain.puts)]:
                if not df.empty:
                    closest_strike = find_closest_value(df['strike'], current_price)
                    filtered = df[df['strike'] == closest_strike].copy()
                    filtered['type'] = opt_type
                    filtered['expiry'] = expiry
                    filtered['symbol'] = symbol
                    options_data.append(filtered)
        if options_data:
            result = pd.concat(options_data, ignore_index=True)
            # Return only strike, impliedVolatility, and openInterest
            return result[['strike', 'impliedVolatility', 'openInterest']]
        else:
            return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest'])
    except Exception as e:
        print(f"Error fetching options for {symbol}: {e}")
        return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest'])
time.sleep(2)



# Example usage:
# filterstocks = ['AAPL', 'MSFT', 'NVDA', ...]
options_list = [get_options(symbol) for symbol in filterstocks] ### Change this to filterstocks to get all options

options = pd.concat(options_list, ignore_index=True)
options = options[options["openInterest"]>100]
options

NameError: name 'filterstocks' is not defined

In [ ]:
## hvol, filterdf, options 
# After filtering for open interest
options = options[options["openInterest"] > 100]

# Calculate average implied volatility for each symbol
result = options.groupby('symbol')['impliedVolatility'].mean().reset_index()

print(result)


    symbol  impliedVolatility
0       AA           0.517241
1      AAL           0.479701
2     AAPL           0.313554
3     ABNB           0.404486
4      ADT           0.383063
..     ...                ...
223   XPEV           0.600071
224    XRX           0.698001
225    YMM           0.495366
226   ZETA           0.675833
227    ZIM           0.855554

[228 rows x 2 columns]


In [ ]:
avg_iv = options.groupby('symbol')['impliedVolatility'].mean().reset_index()
avg_iv.columns = ['symbol', 'avg_iv']  # Rename for clarity
filterdf['symbol'] = filterdf['symbol'].astype(str).str.upper()
avg_iv['symbol'] = avg_iv['symbol'].astype(str).str.upper()
filterdf = filterdf.merge(avg_iv, on='symbol', how='left')

filterdf = filterdf.rename(columns={'hvol': 'historical_volatility', 'avg_iv': 'average_implied_volatility'})
filterdf["iv/hv_ratio"] = filterdf["average_implied_volatility"] / filterdf["historical_volatility"]

import datetime

today = datetime.datetime.today().strftime('%Y-%m-%d')
filename = f'/Users/nshaffer/Desktop/equity_vol_screen_{today}.csv'

filterdf.to_csv(filename, index=False)


filterdf

ValueError: cannot reindex on an axis with duplicate labels